In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/venezia/venezia.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1985.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1996.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1993.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1992.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1986.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_2009.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1988.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_2013.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_2005.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_2001.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1990.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1984.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_2012.csv
/kaggle/input/venezia/Punta_Salute_1983_2015/Punta_Salute_1983.csv
/kaggle/input/venezia/Punta_

In [58]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import tensorflow.keras.backend as K

In [59]:
config = {
    "N_PREV": 90,
    "N_FWD": 30,
    
    "ENCODER_SIZE": 16,
    "DECODER_SIZE": 16,
    
    "TEST_SIZE": 0.1
}

## **Load DataFrame**

In [60]:
df = pd.read_csv('/kaggle/input/venezia/venezia.csv')
df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True)
df.sort_values(by='datetime', inplace=True, ignore_index=True)
df.head()

,datetime,level
0,1983-01-01 01:00:00,44.0
1,1983-01-01 02:00:00,35.0
2,1983-01-01 03:00:00,23.0
3,1983-01-01 04:00:00,10.0
4,1983-01-01 05:00:00,1.0


## **Analying Time Increments**

In [12]:
values = df.datetime.values.astype('int64') // 10**9
ranges = values[1:]-values[:-1]
vals, counts = np.unique(ranges, return_counts=True)
print("TIME INCREMENTS")
for val, count in zip(vals, counts):
    print(f"{val} - {count}")

TIME INCREMENTS
0 - 5
3600 - 289261
7200 - 5


In [13]:
bad_indecies = [[x, x+1] for x in np.where(ranges!=3600)[0]]
print("BAD INDECIES")
for [idx1, idx2] in bad_indecies:
    print(f"INCREMENT: {values[idx2] - values[idx1]} FROM {idx1}: {df.level.values[idx1]} TO {idx2}: {df.level.values[idx1]}")

BAD INDECIES
INCREMENT: 0 FROM 140255: 57.0 TO 140256: 57.0
INCREMENT: 7200 FROM 148991: 6.0 TO 148992: 6.0
INCREMENT: 0 FROM 198815: 54.0 TO 198816: 54.0
INCREMENT: 7200 FROM 198839: 51.0 TO 198840: 51.0
INCREMENT: 0 FROM 219143: 2.0 TO 219144: 2.0
INCREMENT: 7200 FROM 227903: 19.0 TO 227904: 19.0
INCREMENT: 0 FROM 242639: 37.0 TO 242640: 37.0
INCREMENT: 7200 FROM 242663: 56.0 TO 242664: 56.0
INCREMENT: 0 FROM 251399: -1.0 TO 251400: -1.0
INCREMENT: 7200 FROM 251423: 8.0 TO 251424: 8.0


## **Clean Zero Increments**

In [14]:
df = df.drop(axis=0, index=[x[0] for x in bad_indecies if values[x[1]] - values[x[0]] == 0]).reset_index(drop=True)
df.head()

,index,datetime,level
0,0,1983-01-01 01:00:00,44.0
1,1,1983-01-01 02:00:00,35.0
2,2,1983-01-01 03:00:00,23.0
3,3,1983-01-01 04:00:00,10.0
4,4,1983-01-01 05:00:00,1.0


In [15]:
values = df.datetime.values.astype('int64') // 10**9
ranges = values[1:]-values[:-1]
vals, counts = np.unique(ranges, return_counts=True)
print("TIME INCREMENTS")
for val, count in zip(vals, counts):
    print(f"{val} - {count}")

TIME INCREMENTS
3600 - 289261
7200 - 5


In [16]:
bad_indecies = [[x, x+1] for x in np.where(ranges!=3600)[0]]
print("BAD INDECIES")
for [idx1, idx2] in bad_indecies:
    print(f"INCREMENT: {values[idx2] - values[idx1]} FROM {idx1}: {df.level.values[idx1]} TO {idx2}: {df.level.values[idx1]}")

BAD INDECIES
INCREMENT: 7200 FROM 148990: 6.0 TO 148991: 6.0
INCREMENT: 7200 FROM 198837: 51.0 TO 198838: 51.0
INCREMENT: 7200 FROM 227900: 19.0 TO 227901: 19.0
INCREMENT: 7200 FROM 242659: 56.0 TO 242660: 56.0
INCREMENT: 7200 FROM 251418: 8.0 TO 251419: 8.0


## **Data Functions**

In [ ]:
def get_splits():
    data = df.values
    splits = []
    tss = TimeSeriesSplit(n_splits=5, test_size=int(df.shape[0]*config["TEST_SIZE"]))
    for train_idxs, test_idxs in tss.split(data):
        final_train_sample = train_idxs[-1]
        final_test_sample = test_idxs[-1]
        splits.append([final_train_sample, final_test_sample])
    return splits

def get_data(split):
    data = df.values
    X_train, y_train = [], []
    X_val, y_val = [], []
    [final_train_sample, final_test_sample] = split
    for i in range(final_train_sample-config["N_FWD"]-config["N_PREV"]):
        X_train.append(data[i:i+config["N_PREV"]])
        y_train.append(data[i+config["N_PREV"]:i+config["N_PREV"]+config["N_FWD"]])
    for i in range(final_train_sample, final_test_sample-config["N_FWD"]-config["N_PREV"]):
        X_val.append(data[i:i+config["N_PREV"]])
        y_val.append(data[i+config["N_PREV"]:i+config["N_PREV"]+config["N_FWD"]])
    return np.array(X_train), np.array(y_train), np.array(X_val), np.array(y_val)

## **Model Functions**

In [65]:
def simple_encoder_decoder():
    inputs = tf.keras.layers.Input(shape=(config["N_PREV"], 1))
    x, hidden_state, cell_state = tf.keras.layers.LSTM(config["ENCODER_SIZE"], 
                                                       return_sequences=True, 
                                                       return_states=True, 
                                                       recurrent_initializer='glorot_uniform', 
                                                       activity_regularizer=L1L2(l1=0.000001, l2=0.000001))(inputs)
    
    decoder = tf.keras.layers.LSTM(config["DECODER_SIZE"],
                                   return_sequences=True,
                                   return_states=True,
                                   recurrent_initializer='glorot_uniform', 
                                   activity_regularizer=L1L2(l1=0.000001, l2=0.000001))
    decoder_output = tf.keras.layers.Dense(1)
    outputs = []
    last_value = inputs[:, -1, 1]
    states = [hidden_state, cell_state]
    for i in range(config["N_FWD"]):
        x, hidden_state, cell_state = decoder([last_value], initial_state=states)
        states=[hidden_state, cell_state]
        

144636 173563
173563 202490
202490 231417
231417 260344
260344 289271
